<a href="https://colab.research.google.com/github/owenwang05/dish-detect/blob/main/feature_extraction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dish Detect:** Feature-extraction model

**End-to-end CNN Image Classification Model**

Feature extraction model based on EfficientNetB1 trained and evaluated on the Food101 Dataset

## Initial Setup

In [ ]:
# Check the GPU (must be T4 for mixed precision training)
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# Import required packages
import tensorflow as tf
print(tf.__version__)

2.17.0


In [ ]:
# Import addiitonal packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import tensorflow_datasets as tfds
import seaborn as sn

In [ ]:
# Import aelper functions
!wget https://raw.githubusercontent.com/gauravreddy08/deep-learning-tensorflow/main/extras/helper_function.py

--2024-08-20 15:53:05--  https://raw.githubusercontent.com/gauravreddy08/deep-learning-tensorflow/main/extras/helper_function.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_function.py’

helper_function.py  100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-08-20 15:53:05 (58.6 MB/s) - ‘helper_function.py’ saved [10246/10246]



In [ ]:
# Load helper functions
from helper_function import plot_loss_curves, load_and_prep_image

## Prepare Data

The dataset being used is Food101, which is available on TensorFlow

The dataset is imported using the `TensorFlow Datasets` module

In [ ]:
# Check if Food101 dataset is avaliable
datasets_list = tfds.list_builders()
print("food101" in datasets_list)

True


In [ ]:
# Import data set
(train_data, test_data), ds_info = tfds.load(name='food101',
                                             split=['train', 'validation'],
                                             shuffle_files=False,
                                             as_supervised=True,
                                             with_info=True) # provides ds_info variable

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
# Features of Food101 fomr TFDS
ds_info.features

In [ ]:
# Get class names
class_names = ds_info.features["label"].names
class_names[:10]

In [ ]:
# Take on sample of the train data
train_data_sample = train_data.take(1)
train_data_sample

In [ ]:
# Output info about the training sample
for image, label in train_data_sample:
  print(f"Image shape: {image.shape}")
  print(f"Image datatype: {image.dtype}")
  print(f"Target class from Food101: {class_names[label]}")

In [ ]:
# Plot the image
plt.imshow(image)
plt.title(f"Image: {class_names[label]}")
plt.axis(False)

## Preprocess Data

Current Data
* `uint8` datatype
* Different sized tensors
* Unscaled (0-255)

Model Wants
* `float32` datatype
* Same sized tensors
* Normailzied

In [ ]:
# Create preprocessing function
def preprocess_img(image, label, img_size=224):
  image = tf.image.resize(image, [img_size, img_size])
  image = tf.cast(image, tf.float16)
  return image, label

In [ ]:
# Test preprocessing function
preprocessed_img = preprocess_img(image, label)[0]
preprocessed_img

## Batch and Prepare Data

*   `Parallelize` - allows for parallel computing when mapping data to preprocess
*   `Buffer Size` - amount of elements that can be accessed at a time
*   `Prefetch` - prepare second batch while computing patterns on current batch


In [ ]:
train_data = train_data.map(preprocess_img, tf.data.AUTOTUNE)
train_data = train_data.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.AUTOTUNE)

test_data = test_data.map(preprocess_img, tf.data.AUTOTUNE)
test_data = test_data.batch(32)

In [ ]:
train_data

In [ ]:
test_data

## Create Modelling Callbacks

* `Model Checkpoint`- used to save checkpoints during training
* `Early Stopping` - used to stop training when a matric stops improving
* `ReduceLRonPlateau` - used to reduce LR when metric has stopped improving

## Setup Mixed Precision Training

## Build Model

## Evaluate Model

## Save and Export Model